   # Coursework-2 Associated Rule Mining

## Importing Required Libraries

In [1]:
import numpy as np
import pandas as pd
import csv
from itertools import permutations
from itertools import combinations 

In [85]:
# One Hot encoding function to make transaction data into binary
def hot_encode(x):
    if(x<= 0):
        return 0
    if(x>= 1):
        return 1

## Creating a Function for Frequent Itemset Mining

In [3]:
def apriori_freq (data, min_support = 0.04, max_length = 4):
    # Minimum Support is minimum threshold support
    # Maxlength is maximum number of items to be included in itemset
    
    #step-1
    # Creating a dict to store support of an itemset
    support = {}
    L = set(data.columns)
    
    #step-2
    # Generating combination of items with len i in ith iteration
    for i in range(1, max_length+1):
        c = list(combinations(L, i))
        
        # Reset L for next (i+1)th iteration
        L = set()
        
        #step-3
        #iterate through each item in c
        for j in list (c):
            sup = data.loc[:,j].product(axis= 1).sum()/len(data.index)
            if sup > min_support:
                support[j]= sup
                
                # Appending frequent itemset in list L, already reset list L
                L = set(set(L) | set(j))
                
        
    #step-4 Getting results into a df with cols - item & Supp
    result = pd.DataFrame(list(support.items()), columns = ['Items', 'Support'])
    return(result)

### Reference 
    1.PUBLISHED ON NOVEMBER 30, 2020 Guide To Association Rule Mining From Scratch BY ROHIT KRISHNARAO UMREDKAR
    2.Tan, P.N., Steinbach, M. and Kumar, V., 2013. Data mining cluster analysis: basic concepts and algorithms. Introduction to data mining, 487, p.533.
    3.https://github.com/viktree/curly-octo-chainsaw/blob/master/BreadBasket_DMS.csv
    4.https://analyticsindiamag.com/guide-to-association-rule-mining-from-scratch/

## Creating a Function for Association Rule Mining

In [4]:
def rule_association (df, min_threshold = 0.5):
    
    # step-1 
    # Creating  req  variables
    support = pd.Series(df.Support.values, index = df.Items).to_dict()
    data = []
    L = df.Items.values
    
    # step-2
    # Rule generation using permutation
    p = list (permutations(L, 2))
    
    # step-3
    # Iterating through each rule
    for i in p:
        
        # If LHS [Antecedent] of rule is subset of RHS, then valid rule
        if set(i[0]).issubset(i[1]):
            conf = support[i[1]]/support[i[0]]
            # print (i, conf)
            if conf > min_threshold:
                # print (i, conf)
                j = i[1][not i[1].index(i[0][0])]
                lift = support[i[1]]/(support[i[0]]* support[(j,)])
                leverage = support[i[1]] - (support[i[0]]* support[(j,)])
                convection = (1 - support[(j,)])/(1- conf)
                data.append([i[0],(j,), support[i[0]], support[(j,)], support[i[1]], conf, lift, leverage, convection])

    # step-4
    results = pd.DataFrame(data, columns = ['antecedents', 'consequents', 'antecedent_support', 'consequent_support', 
                                           'support', 'confidence', 'lift', 'leverage', 'convection'])
    return(results)
    

### Reference 
    1.PUBLISHED ON NOVEMBER 30, 2020 Guide To Association Rule Mining From Scratch BY ROHIT KRISHNARAO UMREDKAR
    2.Tan, P.N., Steinbach, M. and Kumar, V., 2013. Data mining cluster analysis: basic concepts and algorithms. Introduction to data mining, 487, p.533.
    3.https://github.com/viktree/curly-octo-chainsaw/blob/master/BreadBasket_DMS.csv
    4.https://analyticsindiamag.com/guide-to-association-rule-mining-from-scratch/

## Applying Apriori Algorithm to Small Dataset and Comparing it with External Library

### Loading data

In [5]:
data_small = pd.read_csv('/Users/vinith/tensorflow-test/MSc Data Science/Big Data & Data Mining/Coursework_2/Dataset.Small/GroceryStore.csv', header = None)

In [6]:
data_small.head()

,0
0,"MILK,BREAD,BISCUIT"
1,"BREAD,MILK,BISCUIT,CORNFLAKES"
2,"BREAD,TEA,BOURNVITA"
3,"JAM,MAGGI,BREAD,MILK"
4,"MAGGI,TEA,BISCUIT"


### data preprocessing

In [7]:
df0 = data_small[0].str.split(",", expand = True)
df0

,0,1,2,3
0,MILK,BREAD,BISCUIT,None
1,BREAD,MILK,BISCUIT,CORNFLAKES
2,BREAD,TEA,BOURNVITA,None
3,JAM,MAGGI,BREAD,MILK
4,MAGGI,TEA,BISCUIT,None
5,BREAD,TEA,BOURNVITA,None
6,MAGGI,TEA,CORNFLAKES,None
7,MAGGI,BREAD,TEA,BISCUIT
8,JAM,MAGGI,BREAD,TEA
9,BREAD,MILK,None,None


In [8]:
df0.shape

(20, 4)

In [87]:
# Intializing the list
transactions = []
# Appending the transactions into a list
for i in range(0, 20): 
    transactions.append([str(df0.values[i,j]) for j in range(0, 4)])

In [88]:
transactions

[['MILK', 'BREAD', 'BISCUIT', 'None'],
 ['BREAD', 'MILK', 'BISCUIT', 'CORNFLAKES'],
 ['BREAD', 'TEA', 'BOURNVITA', 'None'],
 ['JAM', 'MAGGI', 'BREAD', 'MILK'],
 ['MAGGI', 'TEA', 'BISCUIT', 'None'],
 ['BREAD', 'TEA', 'BOURNVITA', 'None'],
 ['MAGGI', 'TEA', 'CORNFLAKES', 'None'],
 ['MAGGI', 'BREAD', 'TEA', 'BISCUIT'],
 ['JAM', 'MAGGI', 'BREAD', 'TEA'],
 ['BREAD', 'MILK', 'None', 'None'],
 ['COFFEE', 'COCK', 'BISCUIT', 'CORNFLAKES'],
 ['COFFEE', 'COCK', 'BISCUIT', 'CORNFLAKES'],
 ['COFFEE', 'SUGER', 'BOURNVITA', 'None'],
 ['BREAD', 'COFFEE', 'COCK', 'None'],
 ['BREAD', 'SUGER', 'BISCUIT', 'None'],
 ['COFFEE', 'SUGER', 'CORNFLAKES', 'None'],
 ['BREAD', 'SUGER', 'BOURNVITA', 'None'],
 ['BREAD', 'COFFEE', 'SUGER', 'None'],
 ['BREAD', 'COFFEE', 'SUGER', 'None'],
 ['TEA', 'MILK', 'COFFEE', 'CORNFLAKES']]

In [20]:
from mlxtend.preprocessing import TransactionEncoder

te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df1 = pd.DataFrame(te_ary, columns=te.columns_)
df1 = df1.drop(['None'], axis=1)
df1

,BISCUIT,BOURNVITA,BREAD,COCK,COFFEE,CORNFLAKES,JAM,MAGGI,MILK,SUGER,TEA
0,True,False,True,False,False,False,False,False,True,False,False
1,True,False,True,False,False,True,False,False,True,False,False
2,False,True,True,False,False,False,False,False,False,False,True
3,False,False,True,False,False,False,True,True,True,False,False
4,True,False,False,False,False,False,False,True,False,False,True
5,False,True,True,False,False,False,False,False,False,False,True
6,False,False,False,False,False,True,False,True,False,False,True
7,True,False,True,False,False,False,False,True,False,False,True
8,False,False,True,False,False,False,True,True,False,False,True
9,False,False,True,False,False,False,False,False,True,False,False


### Using Implemented Algorithms

### Frequent Itemset Mining

In [73]:
%timeit -n 100 -r 10 frq_items_small = apriori_freq(df1, min_support = 0.19, max_length = 4)
frq_items_small

19 ms ± 206 µs per loop (mean ± std. dev. of 10 runs, 100 loops each)


,Items,Support
0,"(TEA,)",0.35
1,"(BISCUIT,)",0.35
2,"(MAGGI,)",0.25
3,"(BOURNVITA,)",0.20
4,"(SUGER,)",0.30
5,"(BREAD,)",0.65
6,"(COFFEE,)",0.40
7,"(MILK,)",0.25
8,"(CORNFLAKES,)",0.30
9,"(TEA, MAGGI)",0.20


### Association Rule Mining

In [111]:
%timeit -n 100 -r 10 rules_small = rule_association (frq_items_small, min_threshold = 0.6)
rules_small = rules_small.sort_values(['confidence', 'lift'], ascending =[False, False])
rules_small

285 µs ± 82.9 µs per loop (mean ± std. dev. of 10 runs, 100 loops each)


,antecedents,consequents,antecedent_support,consequent_support,support,confidence,lift,leverage,convection
0,"(MAGGI,)","(TEA,)",0.25,0.35,0.2,0.800000,2.285714,0.1125,3.25
3,"(MILK,)","(BREAD,)",0.25,0.65,0.2,0.800000,1.230769,0.0375,1.75
2,"(SUGER,)","(COFFEE,)",0.30,0.40,0.2,0.666667,1.666667,0.0800,1.80
4,"(CORNFLAKES,)","(COFFEE,)",0.30,0.40,0.2,0.666667,1.666667,0.0800,1.80
1,"(SUGER,)","(BREAD,)",0.30,0.65,0.2,0.666667,1.025641,0.0050,1.05


### Using inbuilt Algorithms

### Frequent Itemset Mining

In [74]:
from mlxtend.frequent_patterns import apriori, association_rules

%timeit -n 100 -r 10 inbuilt_freq_small = apriori(df1, min_support=0.19, max_len=4, use_colnames=True)
inbuilt_freq_small

795 µs ± 119 µs per loop (mean ± std. dev. of 10 runs, 100 loops each)


,support,itemsets
0,0.35,(BISCUIT)
1,0.20,(BOURNVITA)
2,0.65,(BREAD)
3,0.40,(COFFEE)
4,0.30,(CORNFLAKES)
5,0.25,(MAGGI)
6,0.25,(MILK)
7,0.30,(SUGER)
8,0.35,(TEA)
9,0.20,"(BISCUIT, BREAD)"


### Association Rule Mining

In [114]:
%timeit -n 100 -r 10 rules = association_rules(inbuilt_freq_small, min_threshold = 0.6)
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False])
rules

538 µs ± 111 µs per loop (mean ± std. dev. of 10 runs, 100 loops each)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
4,(MAGGI),(TEA),0.25,0.35,0.2,0.800000,2.285714,0.1125,3.25
0,(MILK),(BREAD),0.25,0.65,0.2,0.800000,1.230769,0.0375,1.75
2,(CORNFLAKES),(COFFEE),0.30,0.40,0.2,0.666667,1.666667,0.0800,1.80
3,(SUGER),(COFFEE),0.30,0.40,0.2,0.666667,1.666667,0.0800,1.80
1,(SUGER),(BREAD),0.30,0.65,0.2,0.666667,1.025641,0.0050,1.05


### Reference:
    1.Tan, P.N., Steinbach, M. and Kumar, V., 2013. Data mining cluster analysis: basic concepts and algorithms. Introduction to data mining, 487, p.533.
    2.https://github.com/viktree/curly-octo-chainsaw/blob/master/BreadBasket_DMS.csv
    3.Agrawal, Rakesh, and Ramakrishnan Srikant. "Fast algorithms for mining association rules." Proc. 20th int. conf. very large data bases, VLDB. Vol. 1215. 1994.
    

## Applying Algorithms to France of Large Dataset

### Loading Data

In [26]:
data_large = pd.read_csv('/Users/vinith/tensorflow-test/MSc Data Science/Big Data & Data Mining/Coursework_2/Dataset.Large/OnlineRetail.csv')

In [27]:
data_large.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,01/12/2010 08:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,01/12/2010 08:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,01/12/2010 08:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,01/12/2010 08:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,01/12/2010 08:26,3.39,17850.0,United Kingdom


### Data Pre processing

In [28]:
data_large.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [29]:
# Exploring the different regions of transactions
data_large.Country.unique()

array(['United Kingdom', 'France', 'Australia', 'Netherlands', 'Germany',
       'Norway', 'EIRE', 'Switzerland', 'Spain', 'Poland', 'Portugal',
       'Italy', 'Belgium', 'Lithuania', 'Japan', 'Iceland',
       'Channel Islands', 'Denmark', 'Cyprus', 'Sweden', 'Austria',
       'Israel', 'Finland', 'Bahrain', 'Greece', 'Hong Kong', 'Singapore',
       'Lebanon', 'United Arab Emirates', 'Saudi Arabia',
       'Czech Republic', 'Canada', 'Unspecified', 'Brazil', 'USA',
       'European Community', 'Malta', 'RSA'], dtype=object)

In [30]:
# Stripping extra spaces in the description
data_large['Description'] = data_large['Description'].str.strip()

In [31]:
# Dropping the rows without any invoice number
data_large.dropna(axis = 0, subset =['InvoiceNo'], inplace = True)
data_large['InvoiceNo'] = data_large['InvoiceNo'].astype('str')

In [32]:
# Dropping all transactions which were done on credit
data_large = data_large[~data_large['InvoiceNo'].str.contains('C')]

### Using France Transactions for Association Rule Mining

In [33]:
# Transactions done in France
France = (data_large[data_large['Country'] =="France"]
          .groupby(['InvoiceNo', 'Description'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo'))

In [34]:
France.head()

Description,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 EGG HOUSE PAINTED WOOD,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,12 PENCILS TALL TUBE POSY,12 PENCILS TALL TUBE RED RETROSPOT,12 PENCILS TALL TUBE WOODLAND,...,WRAP VINTAGE PETALS DESIGN,YELLOW COAT RACK PARIS FASHION,YELLOW GIANT GARDEN THERMOMETER,YELLOW SHARK HELICOPTER,ZINC STAR T-LIGHT HOLDER,ZINC FOLKART SLEIGH BELLS,ZINC HERB GARDEN CONTAINER,ZINC METAL HEART DECORATION,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS SMALL
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536370,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536852,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536974,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
537065,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
537463,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [35]:
# Encoding the datasets
France_encoded = France.applymap(hot_encode)
transactions_France = France_encoded
transactions_France.head()

Description,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 EGG HOUSE PAINTED WOOD,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,12 PENCILS TALL TUBE POSY,12 PENCILS TALL TUBE RED RETROSPOT,12 PENCILS TALL TUBE WOODLAND,...,WRAP VINTAGE PETALS DESIGN,YELLOW COAT RACK PARIS FASHION,YELLOW GIANT GARDEN THERMOMETER,YELLOW SHARK HELICOPTER,ZINC STAR T-LIGHT HOLDER,ZINC FOLKART SLEIGH BELLS,ZINC HERB GARDEN CONTAINER,ZINC METAL HEART DECORATION,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS SMALL
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536370,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536852,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536974,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
537065,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
537463,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Using Implemented Function

### Frequent Itemset Mining

In [100]:
%timeit -n 100 -r 10 frq_items_France = apriori_freq(transactions_France, min_support = 0.14, max_length = 4)
frq_items_France

302 ms ± 1.6 ms per loop (mean ± std. dev. of 10 runs, 100 loops each)


,Items,Support
0,"(POSTAGE,)",0.765306
1,"(LUNCH BAG RED RETROSPOT,)",0.153061
2,"(RED TOADSTOOL LED NIGHT LIGHT,)",0.181122
3,"(LUNCH BOX WITH CUTLERY RETROSPOT,)",0.142857
4,"(PLASTERS IN TIN CIRCUS PARADE,)",0.168367
5,"(RABBIT NIGHT LIGHT,)",0.188776
6,"(ROUND SNACK BOXES SET OF4 WOODLAND,)",0.158163
7,"(PLASTERS IN TIN WOODLAND ANIMALS,)",0.170918
8,"(POSTAGE, RED TOADSTOOL LED NIGHT LIGHT)",0.158163
9,"(POSTAGE, ROUND SNACK BOXES SET OF4 WOODLAND)",0.147959


### Association Rule Mining

In [107]:
%timeit -n 100 -r 10 rules_France = rule_association (frq_items_France, min_threshold =0.5)
rules_France

258 µs ± 90.8 µs per loop (mean ± std. dev. of 10 runs, 100 loops each)


,antecedents,consequents,antecedent_support,consequent_support,support,confidence,lift,leverage,convection
0,"(RED TOADSTOOL LED NIGHT LIGHT,)","(POSTAGE,)",0.181122,0.765306,0.158163,0.873239,1.141033,0.019549,1.851474
1,"(PLASTERS IN TIN CIRCUS PARADE,)","(POSTAGE,)",0.168367,0.765306,0.147959,0.878788,1.148283,0.019107,1.936224
2,"(RABBIT NIGHT LIGHT,)","(POSTAGE,)",0.188776,0.765306,0.165816,0.878378,1.147748,0.021345,1.929705
3,"(ROUND SNACK BOXES SET OF4 WOODLAND,)","(POSTAGE,)",0.158163,0.765306,0.147959,0.935484,1.222366,0.026916,3.637755


### Using in built function

### Frequent Itemset Mining

In [102]:
from mlxtend.frequent_patterns import apriori, association_rules

%timeit -n 100 -r 10 inbuilt_freq_large = apriori(transactions_France, min_support=0.14, max_len=4, use_colnames=True)
inbuilt_freq_large

4.3 ms ± 164 µs per loop (mean ± std. dev. of 10 runs, 100 loops each)


,support,itemsets
0,0.153061,(LUNCH BAG RED RETROSPOT)
1,0.142857,(LUNCH BOX WITH CUTLERY RETROSPOT)
2,0.168367,(PLASTERS IN TIN CIRCUS PARADE)
3,0.170918,(PLASTERS IN TIN WOODLAND ANIMALS)
4,0.765306,(POSTAGE)
5,0.188776,(RABBIT NIGHT LIGHT)
6,0.181122,(RED TOADSTOOL LED NIGHT LIGHT)
7,0.158163,(ROUND SNACK BOXES SET OF4 WOODLAND)
8,0.147959,"(PLASTERS IN TIN CIRCUS PARADE, POSTAGE)"
9,0.165816,"(RABBIT NIGHT LIGHT, POSTAGE)"


### Association Rule Mining

In [108]:
%timeit -n 100 -r 10 rules_Fr = association_rules(inbuilt_freq_large, min_threshold = 0.5)
rules_Fr

524 µs ± 118 µs per loop (mean ± std. dev. of 10 runs, 100 loops each)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(PLASTERS IN TIN CIRCUS PARADE),(POSTAGE),0.168367,0.765306,0.147959,0.878788,1.148283,0.019107,1.936224
1,(RABBIT NIGHT LIGHT),(POSTAGE),0.188776,0.765306,0.165816,0.878378,1.147748,0.021345,1.929705
2,(RED TOADSTOOL LED NIGHT LIGHT),(POSTAGE),0.181122,0.765306,0.158163,0.873239,1.141033,0.019549,1.851474
3,(ROUND SNACK BOXES SET OF4 WOODLAND),(POSTAGE),0.158163,0.765306,0.147959,0.935484,1.222366,0.026916,3.637755


### Reference:
    1.https://www.geeksforgeeks.org/implementing-apriori-algorithm-in-python/
    2.Agrawal, Rakesh, and Ramakrishnan Srikant. "Fast algorithms for mining association rules." Proc. 20th int. conf. very large data bases, VLDB. Vol. 1215. 1994.
    3.Tan, P.N., Steinbach, M. and Kumar, V., 2013. Data mining cluster analysis: basic concepts and algorithms. Introduction to data mining, 487, p.533.

## FP-Growth

### Frequent Itemset Mining

In [354]:
class treeNode:
    def __init__(self, nameValue, numOccur, parentNode):
        self.name = nameValue
        self.count = numOccur
        self.nodeLink = None
        self.parent = parentNode      #needs to be updated
        self.children = {} 
#increments the count variable with a given amount    
    def inc(self, numOccur):
        self.count += numOccur
#display tree in text. Useful for debugging        
    def disp(self, ind=1):
        print ('  '*ind, self.name, ' ', self.count)
        for child in self.children.values():
            child.disp(ind+1)

In [355]:
rootNode = treeNode('pyramid',9,None)
rootNode.children['eye'] = treeNode('eye',13,None)
rootNode.disp()

   pyramid   9
     eye   13


In [356]:
def createTree(dataSet, minSup=1): #create FP-tree from dataset but don't mine
    headerTable = {}
    #go over dataSet twice
    for trans in dataSet:#first pass counts frequency of occurance
        for item in trans:
            headerTable[item] = headerTable.get(item, 0) + dataSet[trans]
    for k in list(headerTable):  #remove items not meeting minSup
        if headerTable[k] < minSup: 
            del(headerTable[k])
    freqItemSet = set(headerTable.keys())
    #print 'freqItemSet: ',freqItemSet
    if len(freqItemSet) == 0: return None, None  #if no items meet min support -->get out
    for k in headerTable:
        headerTable[k] = [headerTable[k], None] #reformat headerTable to use Node link 
    #print 'headerTable: ',headerTable
    retTree = treeNode('Null Set', 1, None) #create tree
    for tranSet, count in dataSet.items():  #go through dataset 2nd time
        localD = {}
        for item in tranSet:  #put transaction items in order
            if item in freqItemSet:
                localD[item] = headerTable[item][0]
        if len(localD) > 0:
            orderedItems = [v[0] for v in sorted(localD.items(), key=lambda p: p[1], reverse=True)]
            updateTree(orderedItems, retTree, headerTable, count)#populate tree with ordered freq itemset
    return retTree, headerTable #return tree and header table

In [357]:
def updateTree(items, inTree, headerTable, count):
    if items[0] in inTree.children:#check if orderedItems[0] in retTree.children
        inTree.children[items[0]].inc(count) #incrament count
    else:   #add items[0] to inTree.children
        inTree.children[items[0]] = treeNode(items[0], count, inTree)
        if headerTable[items[0]][1] == None: #update header table 
            headerTable[items[0]][1] = inTree.children[items[0]]
        else:
            updateHeader(headerTable[items[0]][1], inTree.children[items[0]])
    if len(items) > 1:#call updateTree() with remaining ordered items
        updateTree(items[1::], inTree.children[items[0]], headerTable, count)

In [358]:
def updateHeader(nodeToTest, targetNode):   #this version does not use recursion
    while (nodeToTest.nodeLink != None):    #Do not use recursion to traverse a linked list!
        nodeToTest = nodeToTest.nodeLink
    nodeToTest.nodeLink = targetNode

In [45]:
def createInitSet(dataSet):
    retDict = {}
    for trans in dataSet:
        retDict[frozenset(trans)] = 1
    return retDict

In [381]:
def ascendTree(leafNode, prefixPath): #ascends from leaf node to root
    if leafNode.parent != None:
        prefixPath.append(leafNode.name)
        ascendTree(leafNode.parent, prefixPath)

In [382]:
def findPrefixPath(basePat, treeNode): #treeNode comes from header table
    condPats = {}
    while treeNode != None:
        prefixPath = []
        ascendTree(treeNode, prefixPath)
        if len(prefixPath) > 1: 
            condPats[frozenset(prefixPath[1:])] = treeNode.count
        treeNode = treeNode.nodeLink
    return condPats

### Reference:
    1.https://adataanalyst.com/machine-learning/fp-growth-algorithm-python-3/
    2.https://analyticsindiamag.com/introduction-guide-to-fp-tree-algorithm/
    3.Han, J., Pei, J., Yin, Y. and Mao, R., 2004. Mining frequent patterns without candidate generation: A frequent-pattern tree approach. Data mining and knowledge discovery, 8(1), pp.53-87.